<a href="https://colab.research.google.com/github/aghapygad336/MENA/blob/master/gtd_happiness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import ExtraTreesClassifier, AdaBoostRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import random
import pandas as pd
import numpy as np
from openpyxl.utils import dataframe
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import seaborn as sns
from math import sqrt
import matplotlib.pyplot as plt
import math
from matplotlib import colors as cs
from scipy.stats import pearsonr
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import scale
from sklearn import preprocessing, metrics
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, precision_score, recall_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import GridSearchCV

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np

data2015 = pd.read_csv('/content/drive/My Drive/MENA/2015.csv',encoding = "ISO-8859-1", engine = 'python')
data2016 = pd.read_csv('/content/drive/My Drive/MENA/2016.csv',encoding = "ISO-8859-1", engine = 'python')
data2017 = pd.read_csv('/content/drive/My Drive/MENA/2017.csv',encoding = "ISO-8859-1", engine = 'python')
happiness = data2015.append(data2016.append(data2017,sort=True),sort= True)

gtd = pd.read_csv('/content/drive/My Drive/MENA/golbal.csv',encoding = "ISO-8859-1", engine = 'python', usecols=[1,2,3,8,10,19,20,21,22,26,27,29,35,41,71,84,100,101,103,58])
gtd = gtd[(gtd['region_txt'] =='Middle East & North Africa') & (gtd['iyear'] > 2000)]


In [0]:
# gtd missing values
numeric_ = gtd._get_numeric_data()
cat = gtd.select_dtypes(include='object')

mean_nan = numeric_.fillna(numeric_.mean()).dropna(axis=1, how='all')
cat_nan = cat.loc[:, cat.isnull().mean() <0.25]
gtd = mean_nan.join(cat_nan)


In [0]:
# happiness missing values
numeric_ = happiness._get_numeric_data()
cat = happiness.select_dtypes(include='object')

mean_nan = numeric_.fillna(numeric_.mean()).dropna(axis=1, how='all')
cat_nan = cat.loc[:, cat.isnull().mean() <0.25]
happiness = mean_nan.join(cat_nan)

In [0]:
def encoding(dfData):    
    
    df = dfData.copy().select_dtypes(include='object')
    # print(df.columns)
    rows,cols = dfData.shape
    encodings = {}
    for col in df.columns:
        Region_nums = []
        Region_nums = df[col].unique()
        Region_nums = { Region_nums[i]: i for i in range(len(Region_nums)) }
        encodings.update({col:Region_nums})
        dfData[col].replace(Region_nums, inplace=True)
    return dfData,encodings

def get_by_value(dic,value):
    for key, val in dic.items():
        if val == value :
            return key    

In [0]:
gtd, gtd_encodings = encoding(gtd)
happiness, happiness_encodings = encoding(happiness)


In [15]:
get_by_value(gtd_encodings['gname'],67)

'Al-Qaida in the Arabian Peninsula (AQAP)'

In [16]:
top_5 = gtd['gname'].value_counts()[1:6]
top_5 = [203,27,107,67,58]
gtd = gtd[gtd['gname'].isin(top_5)]
gtd

,iyear,imonth,iday,crit1,crit2,crit3,doubtterr,success,suicide,claimed,nkillter,nwound,nwoundte,country_txt,region_txt,attacktype1_txt,targtype1_txt,natlty1_txt,gname,weapsubtype1_txt
72138,2001,4,4,1,1,1,0.0,1,0,0.0,0.0,2.00000,0.000000,7,0,0,7,10,27,15
75727,2003,7,8,1,1,1,0.0,0,0,0.0,0.0,1.00000,0.049719,1,0,3,7,1,27,5
75737,2003,7,10,1,1,1,1.0,1,0,0.0,0.0,1.00000,0.000000,1,0,4,1,1,27,4
75948,2003,10,12,1,1,1,0.0,1,1,1.0,1.0,35.00000,0.000000,7,0,0,5,10,58,6
76090,2003,12,1,1,1,1,0.0,1,0,0.0,0.0,4.00000,0.000000,1,0,0,8,1,27,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181639,2017,12,29,1,1,1,0.0,1,0,0.0,0.0,4.89981,0.000000,7,0,1,1,10,203,2
181670,2017,12,31,1,1,1,0.0,1,0,1.0,0.0,0.00000,0.000000,7,0,1,3,10,203,2
181671,2017,12,31,1,1,1,0.0,1,0,0.0,0.0,9.00000,0.000000,7,0,0,1,10,203,0
181674,2017,12,31,1,1,0,1.0,0,0,0.0,5.0,0.00000,0.000000,7,0,1,8,10,203,2


In [0]:
combinedData=gtd.join(happiness)
combinedData, combinedData_encodings = encoding(combinedData)
happiness, happiness_encodings = encoding(happiness)

In [18]:
combinedData

,iyear,imonth,iday,crit1,crit2,crit3,doubtterr,success,suicide,claimed,nkillter,nwound,nwoundte,country_txt,region_txt,attacktype1_txt,targtype1_txt,natlty1_txt,gname,weapsubtype1_txt,Dystopia Residual,Dystopia.Residual,Economy (GDP per Capita),Economy..GDP.per.Capita.,Family,Freedom,Generosity,Happiness Rank,Happiness Score,Happiness.Rank,Happiness.Score,Health (Life Expectancy),Health..Life.Expectancy.,Lower Confidence Interval,Standard Error,Trust (Government Corruption),Trust..Government.Corruption.,Upper Confidence Interval,Whisker.high,Whisker.low,Country
72138,2001,4,4,1,1,1,0.0,1,0,0.0,0.0,2.00000,0.000000,7,0,0,7,10,27,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75727,2003,7,8,1,1,1,0.0,0,0,0.0,0.0,1.00000,0.049719,1,0,3,7,1,27,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75737,2003,7,10,1,1,1,1.0,1,0,0.0,0.0,1.00000,0.000000,1,0,4,1,1,27,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75948,2003,10,12,1,1,1,0.0,1,1,1.0,1.0,35.00000,0.000000,7,0,0,5,10,58,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76090,2003,12,1,1,1,1,0.0,1,0,0.0,0.0,4.00000,0.000000,1,0,0,8,1,27,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181639,2017,12,29,1,1,1,0.0,1,0,0.0,0.0,4.89981,0.000000,7,0,1,1,10,203,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181670,2017,12,31,1,1,1,0.0,1,0,1.0,0.0,0.00000,0.000000,7,0,1,3,10,203,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181671,2017,12,31,1,1,1,0.0,1,0,0.0,0.0,9.00000,0.000000,7,0,0,1,10,203,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181674,2017,12,31,1,1,0,1.0,0,0,0.0,5.0,0.00000,0.000000,7,0,1,8,10,203,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
combinedData.shape


(9498, 41)

In [0]:
def SplitTrainTes(dataSet):
    train_dataSet=dataSet[(dataSet['iyear'] >=2000) & (dataSet['iyear'] <= 2016)]
    test_dataSet=dataSet[dataSet['iyear'] ==2017]
    return train_dataSet,test_dataSet

In [0]:
train_GTD,test_GTD=SplitTrainTes(gtd)

In [0]:
x_train = train_GTD.drop("gname", axis=1)
y_train = train_GTD["gname"]
x_test = test_GTD.drop("gname", axis=1)
y_test = test_GTD["gname"]

In [0]:
from sklearn.metrics import multilabel_confusion_matrix
def Fmeasure(estimator):
    y_pred_train = estimator.predict(x_train)
    y_pred_test = estimator.predict(x_test)
    print('Train Set Accuracy : ', accuracy_score(y_train, y_pred_train))
    print('Train Set Precision : ', precision_score(y_train, y_pred_train, average='micro'))
    print('Train Set Recall : ', recall_score(y_train, y_pred_train, average='micro'))
    print('Train F-Score for each class : ', f1_score(y_train, y_pred_train, average='micro'))
    print('Train Mean F-Score for both classes : ', f1_score(y_train, y_pred_train, average='micro'))

    print('----------------------------------------------------------------------')
    print('Test Set Accuracy : ', accuracy_score(y_test, y_pred_test))
    print('Test Set Precision : ', precision_score(y_test, y_pred_test, average='micro'))
    print('Test Set Recall : ', recall_score(y_test, y_pred_test, average='micro'))
    print('Test F-Score for each class : ', f1_score(y_test, y_pred_test, average='micro'))
    print('Test Mean F-Score for both classes : ', f1_score(y_test, y_pred_test, average='micro'))

    print('----------------------------------------------------------------------')

In [0]:
def kNearestNeighborsFunction():
    neighbors = list(range(1, 30, 2))
    # empty list that will hold cv scores
    cv_scores = []
    # perform 10-fold cross validation
    for k in neighbors:
        knn = KNeighborsClassifier(n_neighbors=k)
        scores = cross_val_score(knn, x_train, y_train, cv=10, scoring='accuracy')
        cv_scores.append(scores.mean())

    mse = [1 - x for x in cv_scores]
    optimal_k = neighbors[mse.index(min(mse))]
    n=optimal_k
    print('Best N found at ' , n)
    knn = KNeighborsClassifier(n_neighbors=n)
    knn.fit(x_train , y_train)
    Fmeasure(knn)

In [25]:
kNearestNeighborsFunction()

Best N found at  3
Train Set Accuracy :  0.9203720219136196
Train Set Precision :  0.9203720219136196
Train Set Recall :  0.9203720219136196
Train F-Score for each class :  0.9203720219136196
Train Mean F-Score for both classes :  0.9203720219136196
----------------------------------------------------------------------
Test Set Accuracy :  0.8993329290479078
Test Set Precision :  0.8993329290479078
Test Set Recall :  0.8993329290479078
Test F-Score for each class :  0.8993329290479077
Test Mean F-Score for both classes :  0.8993329290479077
----------------------------------------------------------------------


In [0]:

def DT():
    ct_gini = DecisionTreeClassifier()
    ct_entropy = DecisionTreeClassifier(criterion="entropy")
    ct_gini = ct_gini.fit(x_train, y_train)
    ct_entropy = ct_entropy.fit(x_train, y_train)
    Fmeasure(ct_entropy)
    Fmeasure(ct_gini)

In [27]:
DT()

Train Set Accuracy :  0.9998725952350618
Train Set Precision :  0.9998725952350618
Train Set Recall :  0.9998725952350618
Train F-Score for each class :  0.9998725952350618
Train Mean F-Score for both classes :  0.9998725952350618
----------------------------------------------------------------------
Test Set Accuracy :  0.9690721649484536
Test Set Precision :  0.9690721649484536
Test Set Recall :  0.9690721649484536
Test F-Score for each class :  0.9690721649484536
Test Mean F-Score for both classes :  0.9690721649484536
----------------------------------------------------------------------
Train Set Accuracy :  0.9998725952350618
Train Set Precision :  0.9998725952350618
Train Set Recall :  0.9998725952350618
Train F-Score for each class :  0.9998725952350618
Train Mean F-Score for both classes :  0.9998725952350618
----------------------------------------------------------------------
Test Set Accuracy :  0.9630078835657975
Test Set Precision :  0.9630078835657975
Test Set Recall : 

In [0]:
def NB():
    nb = GaussianNB()
    nb = nb.fit(x_train, y_train)
    Fmeasure(nb)


In [29]:
NB()

Train Set Accuracy :  0.39482736654350875
Train Set Precision :  0.39482736654350875
Train Set Recall :  0.39482736654350875
Train F-Score for each class :  0.39482736654350875
Train Mean F-Score for both classes :  0.39482736654350875
----------------------------------------------------------------------
Test Set Accuracy :  0.39357186173438446
Test Set Precision :  0.39357186173438446
Test Set Recall :  0.39357186173438446
Test F-Score for each class :  0.39357186173438446
Test Mean F-Score for both classes :  0.39357186173438446
----------------------------------------------------------------------


In [0]:
def LR():
    grid = {"C": np.logspace(-3, 3, 7), "penalty": ["l1", "l2"]}  # l1 lasso l2 ridge
    logreg = LogisticRegression()
    logreg_cv = GridSearchCV(logreg, grid, cv=10)
    logreg_cv.fit(x_train, y_train)
    print("tuned hpyerparameters :(best parameters) ", logreg_cv.best_params_)
    #Fmeasure(logreg_cv)
    print("accuracy :", logreg_cv.best_score_)

In [0]:
def AB():
    import warnings
    warnings.filterwarnings("ignore")

    param_grid = {
        'learning_rate': [.1, .2, .3, .4, .5],

        'n_estimators': [50, 100, 150, 200, 250]
    }

    classifier = AdaBoostClassifier()
    grid_Search = GridSearchCV(classifier, param_grid=param_grid)
    kk = grid_Search.fit(x_train, y_train)
    Fmeasure(kk)

In [32]:
AB()

Train Set Accuracy :  0.909032997834119
Train Set Precision :  0.909032997834119
Train Set Recall :  0.909032997834119
Train F-Score for each class :  0.909032997834119
Train Mean F-Score for both classes :  0.909032997834119
----------------------------------------------------------------------
Test Set Accuracy :  0.9563371740448757
Test Set Precision :  0.9563371740448757
Test Set Recall :  0.9563371740448757
Test F-Score for each class :  0.9563371740448757
Test Mean F-Score for both classes :  0.9563371740448757
----------------------------------------------------------------------


In [0]:
def SVM():
    from sklearn.svm import SVC
    svclassifier = SVC(kernel='poly', degree=8)
    svclassifier.fit(x_train, y_train)
    Fmeasure(svclassifier)



In [34]:
SVM()

Train Set Accuracy :  0.5441457510510893
Train Set Precision :  0.5441457510510893
Train Set Recall :  0.5441457510510893
Train F-Score for each class :  0.5441457510510893
Train Mean F-Score for both classes :  0.5441457510510893
----------------------------------------------------------------------
Test Set Accuracy :  0.7835051546391752
Test Set Precision :  0.7835051546391752
Test Set Recall :  0.7835051546391752
Test F-Score for each class :  0.7835051546391752
Test Mean F-Score for both classes :  0.7835051546391752
----------------------------------------------------------------------
